# Textklassifikation mit RNN: GermEval 2018

*GermEval* – für German Evaluation – ist ein jährlicher Wettbewerb im Bereich Natural Language Processing für deutschsprachige Texte (s. [https://germeval.github.io/](https://germeval.github.io/)).

Im Jahr 2018 ging es um die Erkennung von Beleidigungen in deutschsprachigen Tweets.

In dieser Aufgabe wollen wir Rekurrente Neuronale Netze (RNN) zur Klassifikation nutzen. Zunächst einmal starten wir mit Vorarbeiten.

## Format der Daten

Die Trainings- und Testdaten liegen als mit Tabulatoren separierte Textdateien (Tab Separated Values – TSV) vor. Uns interessieren die erste Spalte (der Tweet) und die zweite Spalte (`OFFENSE` für Beleidigung bzw. `OTHER` für keine Beleidigung).

In [1]:
! head -10 texts/germeval2018.training.tsv

@corinnamilborn Liebe Corinna, wir würden dich gerne als Moderatorin für uns gewinnen! Wärst du begeisterbar?	OTHER	OTHER
@Martin28a Sie haben ja auch Recht. Unser Tweet war etwas missverständlich. Dass das BVerfG Sachleistungen nicht ausschließt, kritisieren wir.	OTHER	OTHER
@ahrens_theo fröhlicher gruß aus der schönsten stadt der welt theo ⚓️	OTHER	OTHER
@dushanwegner Amis hätten alles und jeden gewählt...nur Hillary wollten sie nicht und eine Fortsetzung von Obama-Politik erst recht nicht..!	OTHER	OTHER
@spdde kein verläßlicher Verhandlungspartner. Nachkarteln nach den Sondierzngsgesprächen - schickt diese Stümper #SPD in die Versenkung.	OFFENSE	INSULT
@Dirki_M Ja, aber wo widersprechen die Zahlen denn denen, die im von uns verlinkten Artikel stehen? In unserem Tweet geht es rein um subs. Geschützte. 2017 ist der gesamte Familiennachzug im Vergleich zu 2016 - die Zahlen, die Hr. Brandner bemüht - übrigens leicht rückläufig gewesen.	OTHER	OTHER
@milenahanm 33 bis 45 habe ich noch gar

## Lesen der Daten

Für das Einlesen der Datensätze verwenden wir die Klasse `NamedTuple`, mit der sich die Daten einfach speichern lassen.

In [ ]:
from collections import namedtuple

Record = namedtuple('Record', [ 'text', 'primary_label', 'secondary_label' ])

with open('texts/germeval2018.training.tsv', 'r') as file:
    training_data = [ Record(*line[:-1].split('\t')) for line in file ]

with open('texts/germeval2018.test.tsv', 'r') as file:
    test_data = [ Record(*line[:-1].split('\t')) for line in file ]

### Aufgabe 1.1 Sichtung der Daten

Geben Sie die ersten fünf Trainingsdatensätze aus.

Welche Besonderheiten fallen Ihnen auf?

In [ ]:
# YOUR CODE HERE 

## Überblick über die Daten

Wir schauen uns die Verteilung der Kategorien in den Trainings- und Testdaten an.

### Aufgabe 1.2 Statistik der Trainings- und Testdaten

Zählen Sie mithilfe der Klasse `Counter` die Beleidigungen in den Trainings- und Testdaten. Unterscheiden sich Test- und Trainingsdaten?

In [ ]:
from collections import Counter

# YOUR CODE HERE

## Installation von SpaCy

Zur Vektorisierung der Texte verwenden wir vortrainierte Word Embeddings von SpaCy.

In [ ]:
!pip install spacy
!python -m spacy download de_core_news_md

## Preprocessing der Tweets

Für die weitere Verarbeitung wollen wir Twitter Handles (`@username`) löschen und das Hashtag-Zeichen entfernen. Damit verhindern wir, dass unser Model später die Namen auswendig lernt, um die Daten zu klassifizieren. 

### Aufgabe 1.3 Aufbereitung der Tweets

Bereiten Sie die Texte wie folgt auf:

- Twitter Handles, d.h. Worte, die mit `@` beginnen, werden entfernt,
- das Hashtag-Zeichen `#` sowie Anführungszeichen werden entfernt,
- Bindestriche `-` werden durch Leerzeichen ersetzt (warum ist das sinnvoll?).

*Tipp: Für die ersten beiden Schritte sind Regular Expressions hilfreich*

In [ ]:
import re

def clean_tweet(text):
    """ Preprocess a tweet. """
    
    # remove handles, i.e. @username
    # remove hashtags, quotes, etc.
    
    # YOUR CODE HERE
    
    return text

clean_tweet(training_data[4].text)

## Vektorisierung mit vortrainierten Wortvektoren

Wir nutzen vortrainierte Wortvektoren aus Spacy.

In [ ]:
import torch
import spacy
import numpy as np

nlp = spacy.load("de_core_news_md")

def vectorize(text):
    """Vectorize text using the German SpaCy tokenizer"""
    return torch.Tensor(np.array([tok.vector for tok in nlp(clean_tweet(text)) if tok.has_vector ]))


### Aufgabe 1.4 Test der Vektorisierung

Vektorisieren Sie den ersten Trainingsdatensatz. Welche Dimension haben die Wortvektoren?

In [ ]:
# YOUR CODE HERE

## Laden der Daten

Mithilfe der Funktion `vectorize()` definieren wir die Funktion `collate_batch()`, die einen Batch in zwei Tensoren – für die Label und die Texte – umwandelt.
Damit wir das RNN später effizient trainieren können, bringen wir die Text-Tensoren mithilfe der Funktion `pad_sequence()` auf die gleiche Länge. 

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

LABEL = { 'OFFENSE': 1, 'OTHER': 0 }

def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    
    for record in batch:
        label_list.append(LABEL[record.primary_label])
        processed_text = vectorize(record.text)
        text_list.append(processed_text)
        lengths.append(processed_text.shape[0])
    return torch.tensor(label_list), pad_sequence(text_list, batch_first=True), lengths

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, num_workers=5, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=5, collate_fn=collate_batch)

### Aufgabe 1.5 Test des DataLoaders

Was gibt der `DataLoader` zurück? Haben die Tensoren bei jedem Batch die gleiche Form? Woran liegt das? 

In [ ]:
# YOUR CODE HERE

## Klassifikation von Text mittels RNNs

Texte bestehen aus einer *Folge* von Wörtern. 
Rekurrente Neuronale Netze (RNNs) eignen sich gut für die Verarbeitung von Folgen.

Unser Netz wird dabei aus zwei Schichten bestehen:
1. das eigentliche RNN aus *Long-Short-Term-Memoy (LSTM)* Zellen oder *Gated Recurrent Units (GRU)*, die die Wortfolge auf eine Folge von *Zuständen* abbilden,
2. einen linearen Layer, der den letzten Zustand auf eine eindimensionale Variable abbildet.


### Aufbau des RNNs

Nun bauen wir das oben beschriebene Netz aus Embedding Layer, RNN Layer und Linear Layer auf.

Die Funktionen `torch.nn.utils.rnn.pack_padded_sequence` und `torch.nn.utils.rnn.pad_packed_sequence` packen bzw. entpacken die Tensoren für eine effiziente Berechnung.
**Wir strukturieren die Daten so, dass Batch die erste Dimension ist (`batch_first = True`)**.

### Aufgabe 1.6 Definition der Netzwerkschichten

Das RNN soll folgende Struktur haben:

- Ein `GRU` mit drei Schichten der Größe `hidden_dim` und dem definierten Dropout,
- ein `Linear` Layer, der die Daten auf zwei Dimensionen reduziert.

**Beachten Sie, dass wir zur Textklassifikation die Ausgabe zum letzten Token verwenden.**

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class RNN(torch.nn.Module) :
    def __init__(self, hidden_dim, embedding_dim = 300, dropout = 0.4) :
        super().__init__()
  
        self.gru = None    # YOUR CODE HERE
        self.linear =None  # YOUR CODE HERE    
        
    def forward(self, _x, **kwargs):
        (x, input_lengths) = _x
        
        x = pack_padded_sequence(x, input_lengths, batch_first=True, enforce_sorted=False)
       
        # Apply GRU
        # YOUR CODE HERE
        
        x, output_lengths = pad_packed_sequence(x, batch_first=True)
        out = None # YOUR CODE HERE 
        return out

## Training und Validierung

### Aufgabe 1.7 Zählen der Parameter

Die Funktion `count_parameters(model)` soll die Zahl der trainierbaren Parameter des Models zurückgeben.


In [ ]:
def count_parameters(model):
    # YOUR CODE HERE
    return 

In [ ]:
model = RNN(hidden_dim=64, dropout=0.5)

count_parameters(model)

### Aufgabe 1.8 Training

Diskutieren sie die folgende Trainings- und Validierungsschleife in ihren Gruppen. Ist die Funktionsweise klar?
Führen Sie das Training für verschiedene Werte von `hidden_dim`, `dropout` und mit unterschiedlichen Lern´raten durch. 

Was ist die beste Accuracy, die Sie erreichen?

### Aufgabe 1.9 Erweiterte Metriken

Bestimmen Sie zusätzlich zur Accuracy *Precision*, *Recall* und *F-Score*.

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer=torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from tqdm.notebook import tqdm

epochs = 10 
training_loss = []
testing_loss = []
training_acc = []
testing_acc = []


with tqdm(range(epochs)) as iterator:
    for epoch in iterator:

        train_loss = 0.0
        train_acc = 0     
        for idx, (target, text, length) in enumerate(train_dataloader):

            target, text = target.to(device), text.to(device)

            optimizer.zero_grad() 
            output = model((text, length))

            loss = loss_fn(output, target) 
            loss.backward() 
            optimizer.step()
            
            train_loss += loss_fn(output, target).item()
            predictions = output.data.max(1)[1]
            train_acc += (predictions == target).sum().item()
 
        training_acc.append(train_acc/len(train_dataloader.dataset))
        training_loss.append(train_loss/len(train_dataloader.dataset))
            
        test_loss = 0
        test_acc = 0
        with torch.no_grad():
            for target, text, length in test_dataloader:
                target, text = target.to(device), text.to(device)
                output = model((text, length))
                loss = loss_fn(output, target)
                prediction = torch.argmax(output, 1)
                test_acc += (prediction == target).sum().item()
                test_loss += loss.item()        
                
            testing_acc.append(test_acc/len(test_dataloader.dataset))
            testing_loss.append(test_loss/len(test_dataloader.dataset))
            
        loss = running_loss/count
        accuracy = 100. * running_correct/count 
        iterator.set_postfix_str(f"train_acc: {train_acc/len(train_dataloader.dataset):.2f} test_acc: {test_acc/len(test_dataloader.dataset):.2f} train_loss: {train_loss/len(train_dataloader.dataset):.2e} test_loss: {test_loss/len(test_dataloader.dataset):.2e}")